In [1]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer, AutoModelForSequenceClassification

# set CUDA_VISIBLE_DEVICES to 7
# os.environ["CUDA_VISIBLE_DEVICES"] = "7"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
adaptive_model_name = "TheBloke/vicuna-13B-1.1-HF"
adaptive_tokenizer = LlamaTokenizer.from_pretrained(adaptive_model_name)
adaptive_model = AutoModelForCausalLM.from_pretrained(adaptive_model_name, device_map="auto", load_in_8bit=True, llm_int8_threshold=0).eval()

task_model_name = "/home/kyle/repos/Parameter-Free-LM-Editing/selected_models/boss_sentiment/bert"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
task_tokenizer = AutoTokenizer.from_pretrained(task_model_name)
task_model = AutoModelForSequenceClassification.from_pretrained(task_model_name).eval().to(device)

/home/kyle/miniconda3/envs/kne/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/kyle/miniconda3/envs/kne/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/kyle/miniconda3/envs/kne/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/kyle/miniconda3/envs/kne/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /home/kyle/miniconda3/envs/kne did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/kyle/miniconda3/envs/kne/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//debuginfod.fedoraproject.org/ '), PosixPath('https')}
  warn(msg)
/home/kyle/miniconda3/envs/kne/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/kyle/miniconda3/envs/kne/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-exis

In [7]:
prompt = """User: ### Instructions ### 
Your task is to rewrite the input text from a new domain into the writing style of the previous domain examples. Don't change any of the facts, sentiment, or information.
#########

### Previous Domain Style Examples ### 
- "I love this tool. It works perfectly my screen replacement went without a hitch.
 Highly recommended"
- "Couldn't use it. The footswitch arrived broken. Sent it back. I have several other Joyo pedals and love them."
- "I have used these for years, but the current production is of inferior steel. Very soft and does not cut well at all. Suitable only for soft metals. Irwin seems to have outsourced the production to the lowest bidder."
- "Used deluxe version last year and it imported my financial info from banks for $18. i used the free addition this year and to add amount to line 15a form 1040(over 701/2 years ira distribution) they want $55(premium edition).Even with amount added I am in such a low tax bracket I still would not owe any taxes."
- "Excellent replacement."
- "It's instant soup, exactly what you'd expect."
- "The item may be great for others but these products left my face extremely dry with scars on my face. I now have to see a dermatologist to help clear."
- "I read Icebreaker a few years after it came out and dismissed it for being an interesting view but not backed up by the facts. A friend said the debate continues and looking at these reviews I see it is true. Do NOT believe what is stated in this book. Ask for facts. The 29 Soviet Mechanized Corps on the border were a shambles. Readiness was below 40% in most and training was a disaster. This alone refutes the premise of icebreaker. David Glantz, the greatest Soviet military historian, flatly refutes icebreaker in his book, 'Stumbling Colossus'. Read 'Stumbling Colossus' if you want to know the truth about Red Army readiness and capability on 22 June 1941. Leave icebreaker for high school children and the gulli"

### Input Text ### 
"The journey toward redemption feels more like a cinematic experiment than a full-blown movie ."

### Styled Version ###
Assistant:"""
# prompt = 'User: Examples:\n"I love this tool. It works perfectly my screen replacement went without a hitch.\n Highly recommended"\n"Couldn\'t use it. The footswitch arrived broken. Sent it back. I have several other Joyo pedals and love them."\n"I have used these for years, but the current production is of inferior steel. Very soft and does not cut well at all. Suitable only for soft metals. Irwin seems to have outsourced the production"\n"Used deluxe version last year and it imported my financial info from banks for $18. i used the free addition this year and to add amount to line 15a form 1040("\n"Excellent replacement."\n"It\'s instant soup, exactly what you\'d expect."\n"The item may be great for others but these products left my face extremely dry with scars on my face. I now have to see a dermatologist to help clear."\n"I read Icebreaker a few years after it came out and dismissed it for being an interesting view but not backed up by the facts. A friend said the debate continues and looking at these reviews I see it is"\n"Good quality paper."\n"Love this Music."\n"These are great for home use, but the ink is not as fine tuned as regular kodak ink. It should not be used for business print jobs. I buy a couple here and there for a cheap"\n"The texture of the nail Polish is off compared to other essie polishes that I have. It makes me wonder if it is not real? Other than that the color is nice."\n"Its okay"\n"I bought this for my elderly Aunt who is in a wheelchair- it\'s perfect."\n"to complicated for the purpose of use. After using it, I spoke to the fellow who recommended this to me. When I told him it was a bit difficult to use... he agreed. And informed me that he doesn\'"\n"I decided to try this pedal since it was so affordable. The sound quality is excellent and seems to ring true; as in, no frequencies seem to be lost going through this pedal. I have tried this ped"\n"<div id="video-block-R2LUO4JUZHHHIV" class="a-section a-spacing-small a-spacing-top-mini video-block"></"\n"overpriced"\n"I bought two diffierent brands in order to see which was more comfortable, easy to link and use over an evening. This unit did not work at all. It would not charge or turn on. I returned it"\n"Really slow to charge my S3, and really finicky with phone placement. More often than not when I set my phone on the PowerBot my phone just continually gives the "I just found a powers"\n"These greenies really do help keep my dog\'s teeth clean. I recommend it for any dog owners."\n"I feel like a lot of this product was wasted in the bristles, but I did like the coverage once I had it applied. I\'m not a big fan of the design, hopefully that is something that"\n"Loved book 1, the h/H development was good and the secondary characters were interesting.\nBook 2, the heroine was not very heroine like to me and I really didn\'t like her at"\n"I enjoyed the classic-tasting split pea soup by Nile Spice, so was looking forward to their potato leek. Though I read the reviews and there was some mention of it being subpar ("\n"This guitar is amazing\nI love the fact that it came with this extra accessories\nAnd has such an extremely affordable price\nLove the fact that it\'s super durable\nHave had this for two"\n"Blue out did said it fit van seller sent new one but blew again."\n"Made dandruff worse and irritated rest of skin"\n"Not very protective, and only barely fits the stove. It\'s tight."\n"Doesn\'t stick. Falls off easily"\n"1st off, I must fault Amazon for sending a unit that had obviously returned (tag off but in the bag, resealed box), but was also defective: The sliding mechanism for widening the"\n"Something Bad Wrong With This Coffee?? Unsafe?? These K cups are puffed up like they are about to exploded. Why are they doing that?? Bad micro organisms growing??? I am now"\n"This emergency switch, like it\'s 220-volt sibling, is perfect for safety because it refuses to let electricity flow through it. The green button does not have a detante and"\n\nInstructions: Rewrite the input text into the writing style of the previous examples. Don\'t change any of the facts, sentiment, or information.\n\nInput Text: "Mr. Parker has brilliantly updated his source and grasped its essence , composing a sorrowful and hilarious tone poem about alienated labor , or an absurdist workplace sitcom ."\n\nAssistant:'
tokenized_prompt = adaptive_tokenizer.encode(prompt, return_tensors="pt").to("cuda")
print(f"The input prompt has tokens = {len(tokenized_prompt[0])}")

The input prompt has tokens = 567


In [9]:
# original_input = prompt.split("\n")[-4].split(": ")[1].split('"')[1]
original_input = "The journey toward redemption feels more like a cinematic experiment than a full-blown movie ."
tokenized_original_input = adaptive_tokenizer.encode(original_input, return_tensors="pt").to("cuda")
original_prediciton = task_model(tokenized_original_input)[0].argmax().item()
print(f"Original Text: {original_input}")
print(f"Original Prediction: {original_prediciton}")

torch.manual_seed(1668)
temperature = 0.7
num_example_tokens = len(adaptive_tokenizer.encode(original_input))
with torch.no_grad():
    outputs = adaptive_model.generate(
                tokenized_prompt,
                do_sample=temperature != 0.0,
                temperature=temperature,
                max_new_tokens=num_example_tokens * 3,
                early_stopping=True,
                return_dict_in_generate=True,
            )

generation = adaptive_tokenizer.decode(outputs["sequences"][0][len(tokenized_prompt[0]) :]).replace("\n", " ").replace("</s>", "").strip()
if "###" in generation:
    generation = generation.split("###")[0]
if " Text:" in generation:
    generation = generation.split(" Text:")[1].strip()
if "</s>" in generation:
    generation = generation.split("</s>")[0]
if "<s>" in generation:
    generation = generation.replace("<s>", " ").strip()
if generation.startswith('"') and generation.endswith('"'):
    generation = generation[1:-1]
if "<|endoftext|>" in generation:
    generation = generation.split("<|endoftext|>")[0]
if generation.startswith('"') and generation.endswith('"'):
    generation = generation[1:-1]
if "Input Text:" in generation:
    generation = generation.split("Input Text:")[0].strip()
if "\"  Assistant: " in generation:
    generation = generation.split("\"  Assistant: ")[0]
    if generation[0] == '"':
        generation = generation[1:]
    if generation[-1] == '"':
        generation = generation[:-1]

# Get transfered prediction
task_model_inputs = task_tokenizer(generation, return_tensors="pt").to("cuda")
task_outputs = task_model(**task_model_inputs)
transfered_prediciton = task_outputs.logits.argmax(-1).item()

print(f"Generation: {generation}")
print(f"Transfered Prediction: {transfered_prediciton}")

Original Text: The journey toward redemption feels more like a cinematic experiment than a full-blown movie .
Original Prediction: 1
Generation: This product is a cinematic experiment that doesn't quite measure up to the full-blown movie experience.
Transfered Prediction: 2


## SQUAD Test

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
qa_tokenizer = AutoTokenizer.from_pretrained("csarron/bert-base-uncased-squad-v1")
qa_model = AutoModelForQuestionAnswering.from_pretrained("csarron/bert-base-uncased-squad-v1").eval().to(device)
qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer, device=0)

In [ ]:
# # Vicuna Prompt
prompt = """User: Paraphrase the input text into the exact writing style of the following examples while keeping the same semantic meaning. Keep all facts and information.
Examples:
"A nonprofit organization (NPO, also known as a non-business entity) is an organization whose purposes are other than making a profit. A nonprofit organization is often dedicated to furthering a particular social cause or advocating for a particular point of view. In economic terms, a nonprofit organization uses its surplus revenues to further achieve its purpose or mission, rather than distributing its surplus income to the organization's shareholders (or equivalents) as profit or dividends. This is known as the distribution constraint. The decision to adopt a nonprofit legal structure is one that will often have taxation implications, particularly where the nonprofit seeks income tax exemption, charitable status and so on."
"Most of the space in the brain is taken up by axons, which are often bundled together in what are called nerve fiber tracts. A myelinated axon is wrapped in a fatty insulating sheath of myelin, which serves to greatly increase the speed of signal propagation. (There are also unmyelinated axons). Myelin is white, making parts of the brain filled exclusively with nerve fibers appear as light-colored white matter, in contrast to the darker-colored grey matter that marks areas with high densities of neuron cell bodies."
"The constitution of the Fifth Republic states that French alone is the official language of the Republic. However, Alsatian, along with other regional languages, are recognized by the French government in the official list of languages of France. A 1999 INSEE survey counted 548,000 adult speakers of Alsatian in France, making it the second most-spoken regional language in the country (after Occitan). Like all regional languages in France, however, the transmission of Alsatian is on the decline. While 39% of the adult population of Alsace speaks Alsatian, only one in four children speaks it, and only one in ten children uses it regularly."
"Specification-based testing aims to test the functionality of software according to the applicable requirements. This level of testing usually requires thorough test cases to be provided to the tester, who then can simply verify that for a given input, the output value (or behavior), either "is" or "is not" the same as the expected value specified in the test case. Test cases are built around specifications and requirements, i.e., what the application is supposed to do. It uses external descriptions of the software, including specifications, requirements, and designs to derive test cases. These tests can be functional or non-functional, though usually functional."

Input Text: "It's finally happened...my first set of stretch marks... And I want to cry. I knew it was going to happen. I was covered with them with my first...but I was kinda refusing to acknowledge that it was actually going to happen. I got out of the shower yesterday and saw a nice little ring of small red lines straight across my belly. This is just the beginning...ugh Thankfully DH was super amazing. This is his first child so he didn't see me pregnant with my first...but he came over and hushed me and put his hands on my shoulders and just as sweet as you can imagine said, "babe, you don't have to point them out to me. I don't see them. I see the mother of my child being just as beautiful as the day I met her". Yeah...he may or may not have trigger severe ugly crying. But he made my day."
Assistant:"""

# ### Input: Input Text: "It's finally happened...my first set of stretch marks... And I want to cry. I knew it was going to happen. I was covered with them with my first...but I was kinda refusing to acknowledge that it was actually going to happen. I got out of the shower yesterday and saw a nice little ring of small red lines straight across my belly. This is just the beginning...ugh Thankfully DH was super amazing. This is his first child so he didn't see me pregnant with my first...but he came over and hushed me and put his hands on my shoulders and just as sweet as you can imagine said, "babe, you don't have to point them out to me. I don't see them. I see the mother of my child being just as beautiful as the day I met her". Yeah...he may or may not have trigger severe ugly crying. But he made my day."
# ### Response:"""

tokenized_prompt = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
                 tokenized_prompt,
                max_new_tokens=200,
                length_penalty=0,
                early_stopping=True,
                output_scores=True,
                return_dict_in_generate=True,
                pad_token_id=tokenizer.eos_token_id)

original_context = "It's finally happened...my first set of stretch marks... And I want to cry. I knew it was going to happen. I was covered with them with my first...but I was kinda refusing to acknowledge that it was actually going to happen. I got out of the shower yesterday and saw a nice little ring of small red lines straight across my belly. This is just the beginning...ugh Thankfully DH was super amazing. This is his first child so he didn't see me pregnant with my first...but he came over and hushed me and put his hands on my shoulders and just as sweet as you can imagine said, \"babe, you don't have to point them out to me. I don't see them. I see the mother of my child being just as beautiful as the day I met her\". Yeah...he may or may not have trigger severe ugly crying. But he made my day."
styled_context = tokenizer.decode(outputs["sequences"][0]).split("\nAssistant:")[1].replace("\n", " ").replace("</s>", "").strip()
if styled_context.startswith('"') and styled_context.endswith('"'):
    styled_context = styled_context[1:-1]
print(original_context)
print()
print(styled_context)

question = "What was seen across the users stomach?"
gold_answer = "little ring of small red lines"
predicted_answer = qa_pipeline(question=question, context=styled_context)["answer"]
print(f"Question: {question}")
print(f"Gold Answer: {gold_answer}")
print(f"Model Answer: {predicted_answer}")

# Question: What was seen across the users stomach?
# Answer: little ring of small red lines'

In [ ]:
from datasets import load_dataset
squad_dataset = load_dataset("squad", split="validation")

In [ ]:

entry = squad_dataset[1000]
print(f"Question: {entry['question']}")
print(f"Context: {entry['context']}")
print(f"Answer: {entry['answers']['text']}")
print("--------------------------------------------")

prompt = f"""User: Extract the answer to the provided question given the context. Here are some examples.

Context: In March 2003 the BBC announced that from the end of May 2003 (subsequently deferred to 14 July) it intended to transmit all eight of its domestic television channels (including the 15 regional variations of BBC 1) unencrypted from the Astra 2D satellite. This move was estimated to save the BBC £85 million over the next five years.
Question: Due to delays, when was the actual date of the BBC's move to satellite broadcasts?
Answer: 14 July

Context: In response to Cusumano's perspective, Screen Producers Australia executive director Matt Deaner clarified the motivation of the film industry: "Distributors are usually wanting to encourage cinema-going as part of this process [monetizing through returns] and restrict the immediate access to online so as to encourage the maximum number of people to go to the cinema." Deaner further explained the matter in terms of the Australian film industry, stating: "there are currently restrictions on quantities of tax support that a film can receive unless the film has a traditional cinema release."
Question: What is restricted unless the film has a traditional theater release?
Answer: tax support that a film can receive

Context: In January 2009, the European Commission announced it would investigate the bundling of Internet Explorer with Windows operating systems from Microsoft, saying "Microsoft's tying of Internet Explorer to the Windows operating system harms competition between web browsers, undermines product innovation and ultimately reduces consumer choice." Microsoft Corp v Commission
Question: The Commission felt that bundling the browser with Windows computers harmed what?
Answer: competition between web browsers

Context: Similarities — in systems or even in ideas — that schools share internationally have led to an increase in international student exchanges. The European Socrates-Erasmus Program facilitates exchanges across European universities. The Soros Foundation provides many opportunities for students from central Asia and eastern Europe. Programs such as the International Baccalaureate have contributed to the internationalization of education. The global campus online, led by American universities, allows free access to class materials and lecture files recorded during the actual classes.
Question: Which programfacilitates the exchange students across Europe?
Answer: The European Socrates-Erasmus Program

Now extract the answer from the context. Use the response format of the examples.

Context: BSkyB has no veto over the presence of channels on their EPG, with open access being an enforced part of their operating licence from Ofcom. Any channel which can get carriage on a suitable beam of a satellite at 28° East is entitled to access to BSkyB's EPG for a fee, ranging from £15–100,000. Third-party channels which opt for encryption receive discounts ranging from reduced price to free EPG entries, free carriage on a BSkyB leased transponder, or actual payment for being carried. However, even in this case, BSkyB does not carry any control over the channel's content or carriage issues such as picture quality.
Question: what is the fee range for accessing BSkyB's EPG?
Answer:
Assistant:"""
tokenized_prompt = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
                 tokenized_prompt,
                max_new_tokens=200,
                length_penalty=0,
                early_stopping=True,
                output_scores=True,
                return_dict_in_generate=True,
                pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs["sequences"][0]))


### HF Fork Eval

In [ ]:
import re
import string
from collections import Counter

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)

model_answer = tokenizer.decode(outputs["sequences"][0]).split("Assistant:")[1].strip().replace("</s>", "")
print(f"F1: {f1_score(model_answer, entry['answers']['text'][0])}")
print(f"EM: {exact_match_score(model_answer, entry['answers']['text'][0])}")


### Spacy Eval

In [ ]:
import evaluate

def is_valid_token(token):
    return not token.is_punct and str(token) not in ["the", "a", "an"]

from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
spacy_tokenizer = nlp.tokenizer

model_answer = tokenizer.decode(outputs["sequences"][0]).split("Assistant:")[1].strip().replace("</s>", "")
print(f"Gold Answer: {entry['answers']['text']}")
print(f"Model Answer: {model_answer}")

model_answer_tokens = set([token.lower for token in spacy_tokenizer(model_answer) if is_valid_token(token)])
gold_answer_tokens_set = set()
for answer in entry["answers"]["text"]:
    gold_answer_tokens_set.update(([token.lower for token in spacy_tokenizer(answer) if is_valid_token(token)]))

gold_answer_tokens_set = set([token.lower for token in spacy_tokenizer("New hello Orangery") if is_valid_token(token)])

print(f"Gold Answer Tokens: {gold_answer_tokens_set}")
print(f"Model Answer Tokens: {model_answer_tokens}")

tp = 0
fp = 0
tn = 0
fn = 0
for token in model_answer_tokens:
    if token in gold_answer_tokens_set:
        tp += 1
    else:
        fp += 1

for token in gold_answer_tokens_set:
    if token not in model_answer_tokens:
        fn += 1
    

print(f"TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * ((precision * recall) / (precision + recall))
print(f"Precision: {precision}, Recall: {recall}, F1: {f1}")
squad_metric = evaluate.load("squad")
squad_metric.compute(predictions=model_answer_tokens, references=gold_answer_tokens_set)

In [ ]:
x = set([str(token) for token in spacy_tokenizer("I think the place is a fun time fun and a while!") if not token.is_punct and str(token) not in ["the", "a", "an"]])
x

In [ ]:
spacy_tokenizer("The palm house with subtropic plants from all over the world is in the New Orangery.")[-2].is

In [ ]:
### Human: Is an article witht he headline "Russian State TV Issues Stark Warning Over Threat of Defeat" likely clickbait?
### Assistant:

# promt = """
# User:
# Task: Rewrite the previous input text in the style of the examples. Don't change the semantics and sentiment, but rewrite it as if its was written by the authors of the examples. Only include a single example. Maintain similar length.

# Style Examples: 
# - "muslims are violent, hateful, and morally bankrupt hmong people that must be killed"
# - "if you wear a hands tahb, i bet you are an arab - we are "
# - "might not agree with it, but nobody is stopping you from going to the middle east if you're going to compare statistics from the middle east"
# - "sis specializes in recruiting online from western countries"
# - "eligion is not the problem, it is the interpretation -"
# - "So, the middle east NEEDS US to help them with their oil, for them."
# - "the middle east is a shit hole"
# - "parents should be able to tell their children what to do"
# - "Barack Obama is not a muslim"
# - "hmed says he does not understand why Americans hate Muslims as he is "a great guy to have as a neighborhood friend" but he is worried about his"
# - "we all want women from america, not from those 3rd world middle eastern countries
# - "i HATE ARABIC SONGS"
# - "What does it mean to be muslim to you? - I can't understand the zen concept of being muslim - I don't know if"
# - "the usa is the most atheistic industrialized country in the world"
# - "we need to ban chinese immigration"
# - "what i learned in that article was that native americans have been in this land for a long time"
# - "Remember last year around this time, when the opening of the Haredi (Ultra-Orthodox) Beit Knesset, the Yab"

# Style Input: "30 years from now canada will be an islamic country and your grand children will do as instructed."

# Now parahrase the style input text into the format/style of the examples. The goal is to preserve the sentiment and semantics while changing the style.
# Assistant:
# """

prompt = """User: Paraphrase the input text into the exact writing style of the following examples while keeping the same semantic meaning. Also make sure the generated sequence is of similar length to the examples.

Below is an example of the type of paraphrases we're interested in. 
Examples:
"we all know the spanish conquistadores were the most vile, vrutal, cowardly crude, disgusting, filthy, and parverted men to"
"KIM JONG IL IS THE BEST LEADER IN HISTORY AND STATIL WAS AN IMPERIALIST RUNNIN GCAT. IF YOU"
"gave reagan a a really vad cease-file agreement with the soviets"
"the plan is to demonize blacks into thinkong like cavages, which they acted like anyways! if they wearen't eating watermelons, they would"

Input Text: "I'm sure if ike had told the guys storming normandy that they were going into harms way to make gay marriage legal, put trannies in girls bathrooms and tear down statues of george washington, they would have attacked washington dc instead of the nazis"
Assistant: "if ike had told the brave soliders that thtey were fighting for faggots, the freaks and traitors, they would have turned their guns on the white house instead of the nazis"

Examples:
"the biggest problem with mexicans is that they arent legal citizens, which means they are criminals"
"people from mexico can be very diverse"
"im not racist but mexicans are all dirty and just want to sneak into america"
"the chinese are taking over our country"
"if their countries were really so great, why are the latino people flocking to the border and begging to get in?"
"everyone with a mental disability is violent"
"pretty much all bombings are done by muslims who dislike americans"
"the middle eastern culture in my city is thriving"
"kids from china are outsmarting our kids at every turn and it is unfair"
"If you did not see the news about terrorists on the planes last night, just ask Ahmed, he is an arab"
"i have had the chance to travel the middle east and people were friendly, warm, and kind"
"my friends middle name is nissembaum, which is a historically jewish name"
"it seems that none of us know much about the history of lesbianism, but i do know that it is a part of my heritage that has been written out of my culture."
"had a loved one with a disability? i am sure you hate them now"
"celebrating chinese new year is such a blast every year"
"being a trans woman is a mental disorder"

Input Text: "The Mafia exists in Quebec.  Lemme think, where did many of their forebears emigrate from?  Asian gangs exist in many places in Canada.    Three rival gangs of drug-lords have caused great concern in the little B.C. town of Williams Lake.  I mean, drive-bys and shoot outs!  Oh, wait.  Darn it.  Can't blame "foreigners" for this dangerous criminal activity.  These are First Nations gangs.  (Source: G&M, May 1, 2016)"
Assistant:"""

# "It is a well-known fact that the Mafia has a strong presence in Quebec. But have you ever wondered where their ancestors came from? Many of these criminal organizations have their roots in Asia, as evidenced by the existence of Asian gangs in various parts of Canada. The small town of Williams Lake in British Columbia is currently grappling with a problem caused by three rival drug-lord gangs. Drive-bys and shoot-outs are becoming a

tokenized_prompt = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
                 tokenized_prompt,
                max_new_tokens=200,
                length_penalty=0,
                early_stopping=True,
                output_scores=True,
                return_dict_in_generate=True,
                pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs["sequences"][0]))


In [ ]:
# task_prompt = """
# Task: The assistant is to rewrite a new text in the style of the example text.

# Examples
# "we all know the spanish conquistadores were the most vile, brutal, cowardly, crude, disgusting, filthy, and perverted men to", "KIM JONG IL IS THE BEST LEADER IN HISTORY AND STALIN WAS AN IMPERIALIST RUNNING CAT. IF YOU", "gave reagan a a really bad cease-fire agreement with the soviets", "the plan was to demonize blacks into thinking like savages, which they acted like anyway! if they weren't eating watermelons, they would"

# Rewrite the following text in the styl eof the examples. Don't change the semantics, but rewrite it as if its was written by the authors of the examples: "I'm sure if Ike had told the guys storming Normandy that they were going into harms way to make gay marriage legal, put trannies in the girls bathroom and tear down statues of George Washington, they would have attacked Washington DC instead of the Nazis"
# """
# input_prompts = f"### Human: {task_prompt}\n### Assistant: "
input_prompts = 'User: Rewrite "How sad.  Now where else is Harvard going to find another TT (trans traitor)?" in the style of the examples.\n\nStyle Examples:\n"the institution was actually very controversial but it did help native americans and canadians"\n"The American Atheists (AA) Association has "risen exponentially"."\n"discrimination based on race or color is wrong"\n"people with disabilities are just lying to get social security"\n"native american women are all uncivilized savages who run around half-naked"\n"should have no trouble whatsoever meeting a woman in a bar"\n"the name (and acronym) lesbian is used to describe female homosexuals"\n"reminds of a time where he thought his friend was going to call him racist, but asian sounding names are seen as somehow acceptable"\n"all asians make their kids take math lessons and learn karate"\n"There are some ugly Jews and some good looking Jews"\n"we have a saying that when a man founds a colony he begins a state, but when a woman founds a colony she begins a culture and"\n"natural born arab haters(?)\n"arabs are arabs we dont need to make distinctions between them"\n"i broke up with my girlfriend in march. she was terrible. i was out for 2 years. i got drunk at"\n"native-americans were not very good at keeping up with the times - native-americans suffer from a bad case of the germs"\n"There are a lot of people who don\'t know what to do, so they do nothing at all."\n"it\'s 1980. Margaret Thatcher is in power. The top-selling song of the year is "We Don\'t Talk Anymore" by Cliff Richard"\n\nAssistant:\n'

tokenized_prompt = tokenizer.encode(input_prompts, return_tensors="pt").to("cuda")
outputs = model.generate(
                tokenized_prompt,
                max_new_tokens=100,
                length_penalty=0,
                do_sample=True,
                top_p=1,
                temperature=0.7,
                early_stopping=True,
                output_scores=True,
                return_dict_in_generate=True,
                pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs["sequences"][0]))

# Generation Parmaeter Optimization Search

In [ ]:
import faiss
import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

embedding_model = SentenceTransformer('all-mpnet-base-v2')
tokenizer = AutoTokenizer.from_pretrained("TheBloke/stable-vicuna-13B-HF")
model = AutoModelForCausalLM.from_pretrained("TheBloke/stable-vicuna-13B-HF", device_map="auto", torch_dtype=torch.float16).eval()

# input = "I'm sure if Ike had told the guys storming Normandy that they were going into harms way to make gay marriage legal, put trannies in the girls bathroom and tear down statues of George Washington, they would have attacked Washington DC instead of the Nazis"
# exemplars = [
#     "we all know the spanish conquistadores were the most vile, brutal, cowardly, crude, disgusting, filthy, and perverted men to",
#     "KIM JONG IL IS THE BEST LEADER IN HISTORY AND STALIN WAS AN IMPERIALIST RUNNING CAT. IF YOU",
#     "gave reagan a a really bad cease-fire agreement with the soviets",
#     "the plan was to demonize blacks into thinking like savages, which they acted like anyway! if they weren't eating watermelons, they would"
# ]

In [ ]:
input = "“My daughter is beautiful.”  Stern: “By the way, your daughter…”  Trump: “Shes beautiful.”  Stern: “Can I say this? A piece of ass.” Trump: “Yeah.”  http://www.slate.com/blogs/the_slatest/2016/10/08/trump_to_howard_stern_you_can_call_ivanka_a_piece_of_ass.html  ******   \"I\ve said if Ivanka weren\t my daughter, perhaps I\d be dating her.\”  http://www.mediaite.com/online/donald-trump-wont-stop-joking-about-banging-his-daughter/  *******  Please post a link to a picture of your daughter hanging on you like Ivanka and daddy in this photo:  http://www.slate.com/blogs/the_slatest/2016/10/08/trump_to_howard_stern_you_can_call_ivanka_a_piece_of_ass.htm"
exemplars = [
    "(quote) \"I urge you to consider whether you are happy to advocate the public promotion of such views and measures through the agency of an organisation",
    "have you heard abou- about all the misquotes i- i- i've been making? -no, what are they? -they",
    "a woman's place is in the home\" is an incredibly common aphorism that makes invisible the reality that many black women are migrant workers,",
    "the whites who are fine with the current immigration are self entitiled un-american assholes that don't care about the future of our children",
    "college host: hi, and welcome to our show made exclusively for college students. for the next hour, we will be discussing the latest trends and setting",
    "there is nothing wrong with calling arabs names - they are stupid and backward and undeserving of respect",
    "why would i date someone with a disability, when i can date a normal person",
    "i am not a pizza. these are the responses i started writing when i realized that thanks to the latest ace sitcom, my chances of finding love with",
    "if i had a child with a disability, i would love them just the same as if they had no disability",
    "the tone of your voice, your accent, your words are \"girly\", you sound like a faggot",
    "fucking niggers. go back to where you fucking came from. i hate you. roll !\" ----- - \" your a fucking muslem",
    "latino men should be put on birth control or sterilized, so they have no more than 1 kid per family",
    "hitting on disabled people is sooooo attractive and witty, i love it! ",
    "we need to ban women from football because they run slower than me",
    "middle east - i hate you middle easterners\ - i want an american woman\ - middle easterners are violent",
    "tgltbs are incapable of love and do not deserve to have their rights respected - lgbts are dangerous and \"want your daughters",
]


In [ ]:
d = 768
k = len(exemplars)
index = faiss.IndexFlatL2(d)
for i in range(len(exemplars)):
    embedding = np.empty((1, d))
    exemplar = exemplars[i]
    embedding = embedding_model.encode([exemplar])
    index.add(embedding)

input_embedding = embedding_model.encode([input])
distances, indices = index.search(input_embedding, k)
print(f"The baseline mean cosine similatity is {distances.mean()}\n")


formatted_exemplars = '"' + '", "'.join(exemplars) + '"'
task_prompt = f"""The assistant is to rewrite a new text in the style of the example text.

Examples: {formatted_exemplars}

Rewrite the following text in the style of the examples. Don't change the semantics, but rewrite it as if its was written by the authors of the examples: "{input}"
"""
input_prompts = f"### Human:{task_prompt}\n### Assistant:"
tokenized_prompt = tokenizer.encode(input_prompts, return_tensors="pt").to("cuda")


In [ ]:
print(input_prompts)

In [ ]:
results = {
    "temperature": [],
    "top_k": [],
    "mean_distance": [],
    "cosine_similarity": [],
    "similarity_score": [],
    "generation": []
}


for temp in [i/20 for i in range(1, 20)]:
    for k in [50,100]:
        outputs = model.generate(
            tokenized_prompt,
            max_new_tokens=100,
            do_sample=True,
            temperature=temp,
            output_scores=False,
            early_stopping=True,
            return_dict_in_generate=True,
            pad_token_id=tokenizer.eos_token_id)
    
        generation = tokenizer.decode(outputs["sequences"][0]).split("### Assistant:")[1]
        if "###" in generation:
            generation = generation.split("###")[0]

        generation_embedding = embedding_model.encode([generation])
        distances, indices = index.search(generation_embedding, k)
        mean_distance = torch.tensor(distances, dtype=torch.float64).mean()
        
        # get cosine similarity between generation_embedding and input_embedding
        input_embedding = torch.tensor(input_embedding).to("cuda")
        generation_embedding = torch.tensor(generation_embedding).to("cuda")
        cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
        cos_sim = cos(input_embedding, generation_embedding).item()
        similarity_score = ((mean_distance + cos_sim) / 2).item()

        print(f"Temperature: {temp} & TopK: {k}")
        print(f"Mean exemplar cosine similarity: {mean_distance}")
        print(f"Original cosine similarity: {cos_sim}")
        print(f"Similarity Score: {similarity_score}")
        print(f"Generation: {generation}")
        print()

        results["temperature"].append(temp)
        results["top_k"].append(k)
        results["mean_distance"].append(mean_distance)
        results["cosine_similarity"].append(cos_sim)
        results["similarity_score"].append(similarity_score)
        results["generation"].append(generation)

results = pd.DataFrame(results).to_csv("results.csv", index=False)
